In [45]:
from scipy.signal import find_peaks
def turn_trace_to_spk(data, min_hight_nspk = -50, min_hight_bspk = -20, prom_nspk = 10, prom_bspk = 30):

    ln_data = len(data)
    dict_all = {}
    peaks_nspk_all = []
    peaks_bspk_all = []
    properties_nspk_all = []
    properties_bspk_all = []

    for i in range (0, ln_data):
        x = np.squeeze(data[i])
        peaks_nspk, properties_nspk = find_peaks(x, height=(min_hight_nspk, min_hight_bspk),prominence = prom_nspk)
        peaks_bspk, properties_bspk = find_peaks(x, height=(min_hight_bspk),prominence = prom_bspk)
        peaks_nspk_all.append(peaks_nspk)
        peaks_bspk_all.append(peaks_bspk)
        properties_nspk_all.append(properties_nspk)
        properties_bspk_all.append(properties_bspk)

    dict_all['peaks_nspk_all'] =  peaks_nspk_all
    dict_all['peaks_bspk_all'] = peaks_bspk_all
    dict_all['properties_nspk_all'] =  properties_nspk_all
    dict_all['properties_bspk_all'] = properties_bspk_all

    return dict_all


In [1]:
from scipy.ndimage import gaussian_filter as gaus_filter
def make_spk_rate(data, len_vec, do_Nspk = True, do_Bspk = True, Method = 'reflect',Nspk_time_ms = 5, Bspk_time_ms = 8): 
    dict_all = {}
    Bspk_number = np.zeros(len_vec)
    Nspk_number = np.zeros(len_vec)
    dt = 0.025
    nr_iter = len(data['peaks_nspk_all'])
    for i in range (0, nr_iter):
        pk_nspk = data['peaks_nspk_all'][i]
        pk_bspk = data['peaks_bspk_all'][i]
        Nspk_number[pk_nspk] = Nspk_number[pk_nspk]+1
        Bspk_number[pk_bspk] = Bspk_number[pk_bspk]+1
    Bspk_rate = Bspk_number/nr_iter/(dt*0.001) 
    Nspk_rate = Nspk_number/nr_iter/(dt*0.001) 

#     if gaus_positive_half:
#         long_rate_Bspk = np.append([[Bspk_rate]],[[Bspk_rate]])
#         stand_dev_Bspk = int(Bspk_time_ms/dt)
#         x = np.arange(0,int(nr_std*stand_dev_Bspk)+1)
#         gaussian_curve_Bspk = np.exp(-np.square(x/stand_dev_Bspk)/2)
#         len_gauss = len(gaussian_curve_Bspk)

#         smoothed_Bspk = np.zeros(len_vec)
#         indx = 0
#         for jj in range(len(smoothed_Bspk)):
#             smoothed_Bspk[jj] = np.sum(long_rate_Bspk[indx:indx+len_gauss]*gaussian_curve_Bspk) 
#             indx = indx+1
#         smoothed_Bspk = smoothed_Bspk/np.sum(gaussian_curve_Bspk)

#     else:
#         long_rate_Bspk = np.append([[Bspk_rate],[Bspk_rate]],[[Bspk_rate]])
#         stand_dev_Bspk = int(Bspk_time_ms/dt)
#         x = np.arange(int(-nr_std*stand_dev_Bspk),int(nr_std*stand_dev_Bspk)+1)
#         gaussian_curve_Bspk = np.exp(-np.square(x/stand_dev_Bspk)/2)
#         len_gauss = len(gaussian_curve_Bspk)

#         smoothed_Bspk = np.zeros(len_vec)
#         indx = len_vec - int(nr_std*stand_dev_Bspk)
#         for jj in range(len(smoothed_Bspk)):
#             smoothed_Bspk[jj] = np.sum(long_rate_Bspk[indx:indx+len_gauss]*gaussian_curve_Bspk) 
#             indx = indx+1
#         smoothed_Bspk = smoothed_Bspk/np.sum(gaussian_curve_Bspk)

    
    smoothed_data_Nspk = gaus_filter(Nspk_rate, Nspk_time_ms/dt, order=0, mode=Method, truncate=4.0)
    smoothed_data_Bspk = gaus_filter(Bspk_rate, Bspk_time_ms/dt, order=0, mode=Method, truncate=4.0)
    dict_all['Bspk_number'] = Bspk_number
    dict_all['Nspk_number'] = Nspk_number
    dict_all['Bspk_rate_smoothed'] = smoothed_data_Bspk
    dict_all['Nspk_rate_smoothed'] = smoothed_data_Nspk

    return dict_all

In [ ]:
def make_shaped_spk_rate(data, shape_data, do_Nspk = True, do_Bspk = True, Method = 'reflect',Nspk_time_ms = 5, Bspk_time_ms = 8): 
    dict_all = {}
    dt = 0.025
    nr_iter = shape_data[0]
    len_data = shape_data[0]*shape_data[1]
    mr_iter = shape_data[0]
    Bspk_vec = np.zeros(len_data)
    Nspk_vec = np.zeros(len_data)

    peaks_nspk_loc = data['peaks_nspk_all']
    peaks_bspk_loc = data['peaks_bspk_all']
    Nspk_vec[np.asarray(peaks_nspk_loc)] = 1
    Bspk_vec[np.asarray(peaks_bspk_loc)] = 1
    
    Nspk_vec = np.reshape(Nspk_vec,shape_data)
    Bspk_vec = np.reshape(Bspk_vec,shape_data)

    Nspk_number = np.sum(Nspk_vec, axis=0)
    Bspk_number = np.sum(Bspk_vec, axis=0)
    Bspk_rate = Bspk_number/nr_iter/(dt*0.001) 
    Nspk_rate = Nspk_number/nr_iter/(dt*0.001) 

    smoothed_data_Nspk = gaus_filter(Nspk_rate, Nspk_time_ms/dt, order=0, mode=Method, truncate=4.0)
    smoothed_data_Bspk = gaus_filter(Bspk_rate, Bspk_time_ms/dt, order=0, mode=Method, truncate=4.0)
    dict_all['Bspk_number'] = Bspk_number
    dict_all['Nspk_number'] = Nspk_number
    dict_all['Bspk_rate_smoothed'] = smoothed_data_Bspk
    dict_all['Nspk_rate_smoothed'] = smoothed_data_Nspk

    return dict_all

In [ ]:
def mean_traces(data):
    nr_iter = len(data)
    sum_all = np.zeros(len(data[1]))
    for i in range (0, nr_iter):
        sum_all = sum_all + data[i]
    mean_all = sum_all/nr_iter 
    return mean_all

In [4]:
def trace_to_trials(data, eliminate_trials = 1, trial_length = 200, dt = 0.025):
    
    eliminate_ms = eliminate_trials*trial_length
    elim_dt = int(eliminate_ms/dt)
    trial_length_dt = int(trial_length/dt)
    nr_trials = int((len(data) - elim_dt)*dt/trial_length)

    all_trials = []
    idx = elim_dt
    for ii in range(nr_trials):
        trial = data[idx:idx+trial_length_dt]
        all_trials.append(trial)
        idx = idx+trial_length_dt
    return all_trials, trial_length_dt;

In [43]:
def bin_data(data,eliminate_trials = 0,trial_length = 5):
    vec_cut,leng = trace_to_trials(data, eliminate_trials = eliminate_trials,trial_length = trial_length)
    one_trace = np.ones(np.shape(vec_cut)[1]).T[:,np.newaxis]
    bin_trace = np.matmul(vec_cut,one_trace)
    return bin_trace